# Introduction to GeoPandas

This notebook is self-contained, and has code used in the "Intro to GeoPandas" video on our [youtube channel](https://www.youtube.com/channel/UC34uByppapyrr-gubZMd9OQ).  Please feel free to reach out with any questions. 

Basis für dieses Notbook ist dieses Video: Geospatial Programming: Intro to GeoPandas 
https://www.youtube.com/watch?v=oFajY3FOq6A

In [1]:
import pandas as pd
import geopandas as gpd
import papermill as pm

In [ ]:
import geopandas as gpd

# Load the energy consumption data for Berlin as a GeoDataFrame
energy_data = gpd.read_file('https://raw.githubusercontent.com/OpenDataBerlin/dataset-energieverbrauch-berlin/master/energieverbrauch.geojson')

# Set the coordinate reference system (CRS) to WGS 84 (EPSG:4326)
energy_data.crs = {'init': 'epsg:4326'}

# Print the first few rows of the energy data
print(energy_data.head())

# Group the energy data by neighborhood and sum the energy consumption
energy_by_neighborhood = energy_data.groupby('gebaeude_typ').sum()

# Print the energy consumption by neighborhood
print(energy_by_neighborhood)

# Plot the energy consumption data on a map of Berlin
energy_data.plot(column='energieverbrauch', cmap='Reds', legend=True)

### Hier ein Einschub für a. den Testeinsatz von Papermill und b. Lesen eines Shape-Files

In [ ]:
pm.inspect_notebook(r"C:\Users\User\Desktop\Workbench\Programmieren\Python\GeoPandas\Geopandas-main\Geopandas-main\Intro to Geopandas\DC Construction.ipynb")

In [ ]:
plz_berlin = gpd.read_file(r"C:\Users\User\Desktop\Workbench\GIS\Tests für OpenData 11.2022\Postleitzahlengebiete_-_Berlin\PLZ_Berlin.shp")

In [ ]:
plz_berlin.sort_values(by=['SHAPE_Area'])

In [ ]:
type(plz_berlin)

In [ ]:
plz_berlin.head(30)

In [ ]:
plz_berlin.crs

In [ ]:
plz_berlin.plot()

## Read in the Neighborhood Laydown

In [ ]:
# Links to pull data from the DC OpenData portal

neighborhood_laydown = "https://opendata.arcgis.com/datasets/f6c703ebe2534fc3800609a07bad8f5b_17.geojson"
# If this url fails, you can download the geojson from here: https://dc.esri.com/datasets/dcdev::dc-neighborhood-clusters-1"

datasets = {
    "2021": "https://opendata.arcgis.com/datasets/da3da798fb4f4f0e9de5c3a17c39351f_10.geojson",
    "2020": "https://opendata.arcgis.com/datasets/ac617c291bbd466bbbea6272f87811d3_8.geojson",
    "2019": "https://opendata.arcgis.com/datasets/107f535e5d3347a8ac1e46dbc13669d4_6.geojson",
    "2018": "https://opendata.arcgis.com/datasets/ca581e1b455a46caa266e3476f8205d2_0.geojson",
    "2017": "https://opendata.arcgis.com/datasets/585c8c3ef58c4f1ab1ddf1c759b3a8bd_39.geojson",
    "2016": "https://opendata.arcgis.com/datasets/2dc1a7dbb705471eb38af39acfa16238_28.geojson", 
    "2015": "https://opendata.arcgis.com/datasets/b3283607f9b74457aff420081eec3190_29.geojson",
    "2014": "https://opendata.arcgis.com/datasets/54b57e15f6944af8b413a5e4f88b070c_13.geojson",
    "2013": "https://opendata.arcgis.com/datasets/3d49e06d51984fa2b68f21eed21eba1f_14.geojson",
    "2012": "https://opendata.arcgis.com/datasets/9cbe8553d4e2456ab6c140d83c7e83e0_15.geojson"
}

In [ ]:
gdf_neigh = gpd.read_file(neighborhood_laydown)

In [ ]:
gdf_neigh.head()

In [ ]:
type(gdf_neigh)

In [ ]:
gdf_neigh.crs

In [ ]:
gdf_neigh.to_crs(3857).area.sum()

In [ ]:
gdf_neigh.plot()

## Read in the Construction Permit Files

In [ ]:
df_list = []
for year, url in datasets.items():  
    gdf = gpd.read_file(url)
    gdf['year'] = year
    
    df_list.append(gdf)

In [ ]:
gdf = pd.concat(df_list)
gdf.shape

## Perform a spatial join to add cluster and neighborhood name to the permit data

In [ ]:
gdf_join = gdf.sjoin(gdf_neigh[['NAME', 'NBH_NAMES', 'geometry']], how="left", predicate="within")

In [ ]:
gdf_join.head(2)

## Create a table of counts per cluster per year

In [ ]:
counts_by_year = gdf_join.value_counts(['year', 'NAME']).to_frame().reset_index().sort_values(['NAME', 'year'])

In [ ]:
counts_by_year.columns = ['year', 'cluster', 'permit_count']
counts_by_year

## Convert that table into one row per cluster, with years as columns

In [ ]:
pivoted = counts_by_year.pivot(index="cluster", columns='year', values='permit_count')

In [ ]:
pivoted_gdf = pd.merge(pivoted, gdf_neigh[['NAME', 'NBH_NAMES', 'geometry']], how="left", left_index=True, right_on="NAME")

In [ ]:
pivoted_gdf = gpd.GeoDataFrame(pivoted_gdf, geometry='geometry', crs=4326)

In [ ]:
type(pivoted_gdf)

## Add the percent change from 2012 to 2021

In [ ]:
pivoted_gdf['perc_change'] = (pivoted_gdf['2021'] / pivoted_gdf['2012']) * 100 - 100

In [ ]:
top_10 = pivoted_gdf.sort_values('perc_change', ascending=False).head(10)

## Plot the ten clusters with the highest 'Percent Change'

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
years = top_10.columns[0:-4]
years

In [ ]:
plt.figure(figsize=(15, 5))

for idx, row in top_10.iterrows():
    plt.plot(years, row[years], label=row['NAME'])
plt.legend()

## Plot all neighborhood clusters as a map

In [ ]:
pivoted_gdf.plot(column='perc_change', cmap='RdYlGn', legend=True, figsize=(10, 10))

# Write to file

In [ ]:
pivoted_gdf.to_file("construction_counts_by_cluster_by_year.geojson", driver="GeoJSON")